In [6]:
#Trevor O'Hearn
#8/4/20
#Written to Quickly Scrape the overview table of the Federal Reserve

import pandas as pd
import requests
from bs4 import BeautifulSoup
import WebScrapingMethods as wsm

In [7]:
base_url = 'https://www.federalreserve.gov/releases/h41/'

#get list for links
soup = wsm.getSoup(base_url)
hrefs = soup.select('.col-xs-1 a')
end_urls = []
for a in hrefs:
    #get links for 2020 and 2019
    if (a.attrs['href'][:4] == '2020' or a.attrs['href'][:4] == '2019'
        or a.attrs['href'][:4] == 'curr'):
        end_urls.append(a.attrs['href'] + '/h41.htm')
    else:
        break

urls = wsm.getLinks(base_url, end_url_list=end_urls)

In [8]:
url = urls[0]
bs = wsm.getSoup(url)
date = bs.select('.H41Release td p')[0].text.strip()
tables = bs.select('table')

In [9]:
bs = wsm.getSoup(urls[0])
table_columns = []
table_rows = []
table_data = {}
features = wsm.cleanFeatures(wsm.getFeatures(bs))
k = 0
failures = []
for table in tables:
    #get dimensions of table
    try:
        
        k += 1
        flag = 0
        if (table.select('th', limit=1)[0].attrs['id'].split('c')[1] == '0'):
            print('actually a table: {}'.format(table.select('th', limit=1)[0].text))
            flag = 1
        across = len(table.select('tr')[-1].select('p')) - 1 #can't count the header column
        down = len(table.select('td')) // across
        bank_bool = False
        if (across > 10): #table of banks
            bank_bool = True
        table_columns.append(across)
        table_rows.append(down)
        
        #get features
        table_num = int(table.select('tr th', limit=1)[0].attrs['id'].strip('t').split('c')[0])
        features[table_num]
        flag = 2
        #get data
        tds = table.select('td')
        data = []
        i = 0
        while i < down:
            text = None
            if (bank_bool): #total is in first column
                text = tds[across * i].text.strip()
            else: #total is last column
                text = tds[across * i + across - 1].text.strip()
            if (len(text) > 0):
                data.append(text)
            i += 1
        table_data[table_num] = dict(zip(features[table_num], data))
        flag = 3
        print('zipped : {}, {}'.format(table_num, flag))
    except:
        print(flag)
        failures.append(k)
        continue
print(failures)
#supposed to fail : 0,3,5, 7, 9, 11, 14, 17, 19 (0 indexed)

#take dictionaries of tables and make a singulare dictionary
all_tables_data = {}
for key in table_data:
    for k in table_data[key]:
        if k in all_tables_data.keys(): #key name already exists
            all_tables_data['{} in {}'.format(k, key)] = table_data[key][k]
        else:
            all_tables_data[k] = table_data[key][k]


#take singular dictionary and create dataframe
df = pd.DataFrame(data=all_tables_data, index=[0])


0
actually a table: Reserve Bank credit, related items, andreserve balances of depository institutions at Federal Reserve Banks
zipped : 2, 3
actually a table: Reserve Bank credit, related items, and reserve balances of depository institutions atFederal Reserve Banks
zipped : 3, 3
0
actually a table: Memorandum item
zipped : 5, 3
0
actually a table: Remaining Maturity
zipped : 7, 3
0
actually a table: Account name
zipped : 9, 3
0
actually a table: Credit Facilities LLCs:
zipped : 11, 3
0
actually a table: Assets, liabilities, and capital
zipped : 13, 3
actually a table: Assets, liabilities, and capital
zipped : 14, 3
0
actually a table: Assets, liabilities, and capital
zipped : 16, 3
actually a table: Assets, liabilities, and capital
zipped : 17, 3
0
0
actually a table: Federal Reserve notes and collateral
zipped : 20, 3
0
[1, 4, 6, 8, 10, 12, 15, 18, 19, 21]


In [ ]:
tds = tables[15].select('td')
#get data
data = []
i = 0
while i < down:
    text = tds[across * i + across - 1].text.strip()
    if (len(text) > 0):
        data.append(text)
    i += 1

In [149]:
df.head()

Reserve Bank credit Securities held outright1 U.S. Treasury securities  \
0           6,969,543                 6,126,948                4,213,220   

    Bills2 Notes and bonds, nominal2 Notes and bonds, inflation-indexed2  \
0  326,044                 3,581,139                             270,206   

  Inflation compensation3 Federal agency debt securities2  \
0                  35,831                           2,347   

  Mortgage-backed securities4  \
0                   1,911,381   

  Unamortized premiums on securities held outright5  ...  \
0                                           314,324  ...   

  Less: Notes held by F.R. Banks not subject to collateralization  \
0                                            153,385                

  Federal Reserve notes to be collateralized  \
0                                  1,922,987   

  Collateral held against Federal Reserve notes  \
0                                     1,922,987   

  Gold certificate account in 19  \
0                         11,037   

  Special drawing rights certificate account in 19  \
0                                            5,200   

  U.S. Treasury, agency debt, and mortgage-backed securities pledged1,2  \
0                                          1,906,750                      

  Other assets pledged      Memo:  \
0                    0  6,188,149   

  Total U.S. Treasury, agency debt, and mortgage-backed securities1,2  \
0                                            214,552                    

  Less: Face value of securities under reverse repurchase agreements  
0                                          5,973,597                  

[1 rows x 162 columns]

In [142]:
all_tables_data = {}
for key in table_data:
    for k in table_data[key]:
        if k in all_tables_data.keys(): #key name already exists
            all_tables_data['{} in {}'.format(k, key)] = table_data[key][k]
        else:
            all_tables_data[k] = table_data[key][k]

In [143]:
len(all_tables_data)

162

In [129]:
test_dict = {'first' : 1, 'third' : 3}


In [140]:
if 'second' in test_dict.keys():
    print(True)
else:
    print(False)

False


In [121]:
dfs[0].shape, dfs[1].shape

((1, 35), (1, 14))

In [124]:
dfs[0].columns, dfs[1].columns

(Index(['Reserve Bank credit', 'Securities held outright1',
        'U.S. Treasury securities', 'Bills2', 'Notes and bonds, nominal2',
        'Notes and bonds, inflation-indexed2', 'Inflation compensation3',
        'Federal agency debt securities2', 'Mortgage-backed securities4',
        'Unamortized premiums on securities held outright5',
        'Unamortized discounts on securities held outright5',
        'Repurchase agreements6', 'Foreign official', 'Others', 'Loans',
        'Primary credit', 'Secondary credit', 'Seasonal credit',
        'Primary Dealer Credit Facility',
        'Money Market Mutual Fund Liquidity Facility',
        'Paycheck Protection Program Liquidity Facility',
        'Other credit extensions',
        'Net portfolio holdings of Commercial Paper Funding Facility II LLC7',
        'Net portfolio holdings of Corporate Credit Facilities LLC7',
        'Net portfolio holdings of MS Facilities LLC (Main Street Lending Program)7',
        'Net portfolio holdings

In [113]:
suma

162

In [99]:
table_data

{2: {'Reserve Bank credit': '6,969,543',
  'Securities held outright1': '6,126,948',
  'U.S. Treasury securities': '4,213,220',
  'Bills2': '326,044',
  'Notes and bonds, nominal2': '3,581,139',
  'Notes and bonds, inflation-indexed2': '270,206',
  'Inflation compensation3': '35,831',
  'Federal agency debt securities2': '2,347',
  'Mortgage-backed securities4': '1,911,381',
  'Unamortized premiums on securities held outright5': '314,324',
  'Unamortized discounts on securities held outright5': '-5,345',
  'Repurchase agreements6': '61,201',
  'Foreign official': '1,001',
  'Others': '60,200',
  'Loans': '97,133',
  'Primary credit': '5,860',
  'Secondary credit': '0',
  'Seasonal credit': '16',
  'Primary Dealer Credit Facility': '2,486',
  'Money Market Mutual Fund Liquidity Facility': '20,637',
  'Paycheck Protection Program Liquidity Facility': '68,133',
  'Other credit extensions': '0',
  'Net portfolio holdings of Commercial Paper Funding Facility II LLC7': '12,799',
  'Net portf

In [17]:
len(tables[1].select('tr td')) // across

SelectorSyntaxError: Invalid character '<' position 3
  line 1:
tr < td
   ^ (<string>)

In [12]:

dfs = []
for url in urls:
    bs = wsm.getSoup(url)
    date = bs.select('.H41Release td p')[0].text.strip()
    print(date)
    features = wsm.getFeatures(bs)
    table = bs.select('table')[1]
    tds = table.select('td')
    data = []
    i = 0
    while i < (len(tds) / 4):
        text = tds[4 * i + 3].text.strip()
        if (len(text) > 0):
            data.append(text)
        i += 1
    cleaned_features = wsm.cleanFeatures(features)

    dfvals = {}
    dfvals['Date'] = date
    for i, f in enumerate(cleaned_features[2]):
        dfvals[f] = data[i]

    df = pd.DataFrame(data=dfvals, index=[0])
    dfs.append(df)

#clean dataframe
df = dfs[0]
for d in dfs[1:]:
    df = df.append(d, ignore_index=False)
df['Date'] = pd.to_datetime(df['Date'], format='%B %d, %Y', errors='ignore')
df.set_index('Date', inplace=True)

all_dfs = [df]
columnNames = df.columns
df.fillna('0', inplace=True)
#clean data frame
for df in all_dfs:
    for c in columnNames:
        df[c] = df[c].apply(wsm.removeUnicode)
        df[c] = df[c].apply(wsm.removePlus)
        df[c] = df[c].apply(wsm.removeComma)
        df[c] = df[c].astype(int)
#df_all.to_csv('quickscrape.csv')


July 2, 2020
July 9, 2020
July 16, 2020
July 23, 2020
July 30, 2020
June 4, 2020
June 11, 2020
June 18, 2020
June 25, 2020
May 7, 2020
May 14, 2020
May 21, 2020
May 28, 2020
April 2, 2020
April 9, 2020
April 16, 2020
April 23, 2020
April 30, 2020
March 5, 2020
March 12, 2020
March 19, 2020
March 26, 2020
February 6, 2020
February 13, 2020
February 20, 2020
February 27, 2020
January 2, 2020
January 9, 2020
January 16, 2020
January 23, 2020
January 30, 2020
December 5, 2019
December 12, 2019
December 19, 2019
December 26, 2019
November 7, 2019
November 14, 2019
November 21, 2019
November 29, 2019
October 3, 2019
October 10, 2019
October 17, 2019
October 24, 2019
October 31, 2019
September 5, 2019
September 12, 2019
September 19, 2019
September 26, 2019
August 1, 2019
August 8, 2019
August 15, 2019
August 22, 2019
August 29, 2019
July 5, 2019
July 11, 2019
July 18, 2019
July 25, 2019
June 6, 2019
June 13, 2019
June 20, 2019
June 27, 2019
May 2, 2019
May 9, 2019
May 16, 2019
May 23, 2019
M

In [2]:
dfvals

{'Date': 'July 2, 2020',
 'Reserve Bank credit': '6,969,543',
 'Securities held outright1': '6,126,948',
 'U.S. Treasury securities': '4,213,220',
 'Bills2': '326,044',
 'Notes and bonds, nominal2': '3,581,139',
 'Notes and bonds, inflation-indexed2': '270,206',
 'Inflation compensation3': '35,831',
 'Federal agency debt securities2': '2,347',
 'Mortgage-backed securities4': '1,911,381',
 'Unamortized premiums on securities held outright5': '314,324',
 'Unamortized discounts on securities held outright5': '-5,345',
 'Repurchase agreements6': '61,201',
 'Foreign official': '1,001',
 'Others': '60,200',
 'Loans': '97,133',
 'Primary credit': '5,860',
 'Secondary credit': '0',
 'Seasonal credit': '16',
 'Primary Dealer Credit Facility': '2,486',
 'Money Market Mutual Fund Liquidity Facility': '20,637',
 'Paycheck Protection Program Liquidity Facility': '68,133',
 'Other credit extensions': '0',
 'Net portfolio holdings of Commercial Paper Funding Facility II LLC7': '12,799',
 'Net portfol

In [4]:
df = pd.DataFrame(data=dfvals, index=[0])

In [7]:
df2 = df.copy()

In [9]:
type(df)

dict

In [10]:
dfs

[{'Date': 'July 2, 2020',
  'Reserve Bank credit': '6,969,543',
  'Securities held outright1': '6,126,948',
  'U.S. Treasury securities': '4,213,220',
  'Bills2': '326,044',
  'Notes and bonds, nominal2': '3,581,139',
  'Notes and bonds, inflation-indexed2': '270,206',
  'Inflation compensation3': '35,831',
  'Federal agency debt securities2': '2,347',
  'Mortgage-backed securities4': '1,911,381',
  'Unamortized premiums on securities held outright5': '314,324',
  'Unamortized discounts on securities held outright5': '-5,345',
  'Repurchase agreements6': '61,201',
  'Foreign official': '1,001',
  'Others': '60,200',
  'Loans': '97,133',
  'Primary credit': '5,860',
  'Secondary credit': '0',
  'Seasonal credit': '16',
  'Primary Dealer Credit Facility': '2,486',
  'Money Market Mutual Fund Liquidity Facility': '20,637',
  'Paycheck Protection Program Liquidity Facility': '68,133',
  'Other credit extensions': '0',
  'Net portfolio holdings of Commercial Paper Funding Facility II LLC7':

In [19]:
columns = all_dfs[0].columns

In [21]:
columns

Index(['Bills2', 'Central bank liquidity swaps8',
       'Federal agency debt securities2', 'Float',
       'Foreign currency denominated assets10', 'Foreign official',
       'Gold stock', 'Inflation compensation3', 'Loans',
       'Money Market Mutual Fund Liquidity Facility',
       'Mortgage-backed securities4',
       'Net portfolio holdings of Commercial Paper Funding Facility II LLC7',
       'Net portfolio holdings of Corporate Credit Facilities LLC7',
       'Net portfolio holdings of Corporate Credit Facility LLC7',
       'Net portfolio holdings of MS Facilities LLC (Main Street Lending Program)7',
       'Net portfolio holdings of Maiden Lane LLC7',
       'Net portfolio holdings of Municipal Liquidity Facility LLC7',
       'Net portfolio holdings of TALF II LLC7',
       'Notes and bonds, inflation-indexed2', 'Notes and bonds, nominal2',
       'Other Federal Reserve assets9', 'Other credit extensions', 'Others',
       'Paycheck Protection Program Liquidity Facility',
  

In [25]:
columns[18]

'Notes and bonds, inflation-indexed2'